# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [7]:
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
# load in the csv from part 1 as a dataframe
df = pd.read_csv("weather_data.csv")
df.head()

,City,Latitude,Longitude,Max Temperature,Humidity,Windspeed,Cloudiness,Date,Country
0,Saint Paul Harbor,57.79,-152.41,46.400,61,2.10,90,1602974083,US
1,Arlit,18.74,7.39,90.806,12,3.04,0,1602974132,NE
2,New Norfolk,-42.78,147.06,51.008,96,3.13,59,1602974132,AU
3,Plettenberg Bay,-34.05,23.37,59.738,87,4.19,96,1602974133,ZA
4,Clyde River,70.47,-68.59,37.400,74,7.70,75,1602974133,CA


In [3]:
# configure gmaps
gmaps.configure(api_key=g_key)

In [30]:
# set up plot
figure_layout = {
    'width': '500px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# Create heat layer
heat_layer = gmaps.heatmap_layer(df[["Latitude", "Longitude"]], 
                                 weights=df["Humidity"], 
                                 max_intensity=100) 

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [36]:
# filter down to ideal conditions
filtered_df = df[df["Max Temperature"] < 70]
filtered_df = filtered_df[filtered_df["Max Temperature"] > 65]

filtered_df = filtered_df[filtered_df["Humidity"] < 75]
filtered_df = filtered_df[filtered_df["Humidity"] > 20]

filtered_df = filtered_df[filtered_df["Windspeed"] < 8]
filtered_df = filtered_df[filtered_df["Windspeed"] > 2]
hotel_df = filtered_df[filtered_df["Cloudiness"] < 50]

# check number of resulting ideal cities
len(hotel_df)

10

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
# make list of (lat, long) pairs
lats = hotel_df['Latitude'].tolist()
lngs = hotel_df['Longitude'].tolist()
locs = list(zip(*[lats, lngs]))

In [15]:

hotels = []
for i in range(len(hotel_df)):
    coordinates = f"{locs[i][0]}, {locs[i][1]}"
    target_search = "hotel"
    radius = 5000

    # set up a parameters dictionary
    params = {
        "location": coordinates,
        "keyword": target_search,
        "radius": radius,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    if response.status_code > 399:
        print(response.status_code)
        break
    
    hotels.append(response.json())

# now we have a list of hotel results for each location

In [16]:
# get the first hotel name for each place
names = [h['results'][0]['name'] for h in hotels]
    
# put names into df
hotel_df["Hotel Name"] = names
hotel_df

<ipython-input-16-0adce12a74b9>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = names


,City,Latitude,Longitude,Max Temperature,Humidity,Windspeed,Cloudiness,Date,Country,Hotel Name
31,Turaif,31.67,38.66,64.400,34,2.10,0,1602974139,SA,Golden Dune Hotel
54,Riyadh,24.69,46.72,66.200,25,1.00,0,1602973874,SA,Best Western Plus Fursan Hotel
74,Salalah,17.02,54.09,73.400,64,1.50,0,1602974151,OM,Grand flora hotel salalah
76,Abu Kamal,34.45,40.92,68.666,31,2.74,0,1602974152,SY,مطعم السلام
93,Lüderitz,-26.65,15.16,65.768,45,7.97,0,1602973980,NaN,LÜDERITZ NEST HOTEL
145,Kaeo,-35.10,173.78,64.994,56,3.13,0,1602974169,NZ,Pacific Harbour Lodge
309,Springbok,-29.66,17.89,61.142,36,1.88,0,1602974213,ZA,Springbok Inn by Country Hotels
376,Tezu,27.92,96.17,73.922,66,1.36,0,1602974229,IN,Capital O 66991 Hotel Highway
384,Mhlume,-26.03,31.85,71.186,66,3.06,0,1602974231,SZ,Mhlume Country Club & Lodge
393,Yulara,-25.24,130.99,68.000,52,7.70,0,1602974234,AU,Sails in the Desert


In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [22]:
# create symbol layer
symbols = gmaps.symbol_layer(locations,
                             info_box_content=hotel_info,
                             fill_color='rgba(0, 150, 0, 0.4)',
                             stroke_color='rgba(0, 0, 150, 0.4)', scale=2,)
# Add the layer to the map
fig.add_layer(symbols)

# show figure
fig

Figure(layout=FigureLayout(height='420px'))